In [4]:
import os

import matplotlib.pyplot as plt
import math
import networkx as nx
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors
from sklearnex import patch_sklearn
from tqdm.notebook import tqdm

patch_sklearn()

import tensorflow as tf
from tensorflow import keras
from keras import layers
# from keras.wrappers.scikit_learn import KerasClassifier
from scikeras.wrappers import KerasClassifier

from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import cross_val_score, cross_val_predict, cross_validate, KFold, RepeatedKFold
from sklearn.preprocessing import LabelBinarizer
from utils import OneHotEncoder
from ultility_functions import OneCycleScheduler


print(tf.__version__)
print(tf.keras.__version__)
random_state = 2111

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


ModuleNotFoundError: No module named 'seaborn'

In [4]:
df = pd.read_csv('./slac/Contents of SLAC/slac.csv', usecols=['id','genre', 'subgenre', 'title'])
ids = df.id.tolist()
path = './slac/embeddings/'
emb = KeyedVectors.load_word2vec_format(path+'slac.bin')
data = np.array([emb[i] for i in ids])
emb_sets = {
    'ALL' : data,
}

SyntaxError: invalid syntax (3942081228.py, line 2)

In [95]:
genre = np.array(df.genre).reshape(-1, 1)

lb = LabelBinarizer()
lb_labels = lb.fit_transform(genre)

num_input, num_output = data.shape[1], lb_labels.shape[1]

print(f'genre.shape={genre.shape}',
      f'lb.classes_={lb.classes_}',
      f'data.shape={data.shape}',
      f'lb_labels.shape={lb_labels.shape}',
      f'(num_input, num_output)={(num_input, num_output)}', sep='\n')


genre.shape=(250, 1)
lb.classes_=['Blues' 'Classical' 'Jazz' 'Rap' 'Rock']
data.shape=(250, 100)
lb_labels.shape=(250, 5)
(num_input, num_output)=(100, 5)


In [100]:
def create_n_model(n_layers=1, kernel_init:str=None, activation_fn:str="relu"):
    model = keras.Sequential()
    # Input Layer
    model.add(layers.Dense(100, activation=activation_fn, kernel_initializer=kernel_init, input_shape=(num_input,)))
    # Hidden Layers
    for _ in range(n_layers):
        model.add(layers.Dense(100, activation=activation_fn, kernel_initializer=kernel_init))
    # Output Layers
    model.add(layers.Dense(num_output, activation='sigmoid'))

    model.compile(optimizer=keras.optimizers.Nadam(learning_rate=.01),
                  loss=keras.losses.CategoricalCrossentropy(from_logits=False),
                  metrics=[keras.metrics.CategoricalAccuracy(name='accuracy')])

    return model

In [101]:
net_params = {"n_layers" : 1, "kernel_init" : "he_normal", "activation_fn" : "selu", "epochs" : 10, "batch_size" : 50}

onecycle = OneCycleScheduler(math.ceil(len(data) / net_params["batch_size"]) * net_params["epochs"], max_rate=0.05)

fit_params = {"callbacks" : onecycle}

In [102]:
neural_network = KerasClassifier(model=create_n_model,
                                 verbose=0,
                                 **net_params)


scores = cross_val_score(neural_network, data, lb_labels,
                         cv=KFold(10, shuffle=True, random_state=random_state), n_jobs=-1, verbose=1,
                         fit_params=fit_params)

print(scores)
m = np.mean(scores)
s = np.std(scores)
print('Accuracy: %.3f (%.3f)' % (m, s))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


[0.92 0.84 0.92 0.8  0.84 0.96 0.84 0.88 0.68 0.8 ]
Accuracy: 0.848 (0.075)


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    8.4s finished


In [51]:
scores_5c = {}
scores_5c_all = {}
for name, d in tqdm(emb_sets.items()):
    print(name)
    scores = cross_val_score(neural_network, d, lb_labels,
                             cv=KFold(10, shuffle=True, random_state=random_state), n_jobs=-1, verbose=0)
    print(scores)
    scores_5c[name] = np.mean(scores)
    scores_5c_all[name] = scores

scores_5c

  0%|          | 0/1 [00:00<?, ?it/s]

ALL
[0.16 0.12 0.08 0.08 0.   0.08 0.04 0.08 0.12 0.12]


{'ALL': 0.08800000000000001}